# Actividad De Reposición
#### Stefano Aragoni - 20261

En este Jupyter Notebook se busca poder analizar y explorar un dataset que contiene información médica de pacientes. Esto con el propósito de determinar, a través de un modelo de regresión logística polinomial, si un paciente si un paciente tendrá o no un paro cardiaco. 

_______________

## Tarea 1.1
#### Leer el archivo CSV proporcionado y almacenarlo en un np.array para ser trabajado en el notebook.

Como primer paso, se importaron las librerías a utilizar para el respectivo análisis y creación de modelo.

In [341]:
# Importar librerías 
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import PolynomialFeatures
import matplotlib.pyplot as plt
import seaborn as sns
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import MinMaxScaler
from imblearn.under_sampling import RandomUnderSampler
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import cross_val_score

A continuación, justo como indican las instrucciones, se leyó el archivo CSV y se almacenó en un dataset de pandas. (En las instrucciones decía np.array pero Alberto dijo que con Pandas estaba bien)

In [288]:
# Importar dataset
dataset = pd.read_csv('framingham.csv')

dataset.head()

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,3.0,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,3.0,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0


Posteriormente, se realizó un pequeño análisis exploratorio para determinar si hacen falta datos y si los datos en sí están balanceados. Asimismo, se analizó las mejores variables para crear el respectivo modelo.

##### Datos Faltantes
Como se puede observar a continuación, sí existen unos datos faltantes. Por tal razón, se rellenará dichos espacios con el promedio del feature. 
> Usar el promedio puede presentar problemas, sin embargo, no es viable borrar todos los datos debido a que sería más del 5% de las filas. 

In [289]:
# Matriz de datos faltantes con np.array
missing_values = np.array(dataset.isnull())

# Mostrar columnas con datos faltantes 
dataset.isnull().sum()

male                 0
age                  0
education          105
currentSmoker        0
cigsPerDay          29
BPMeds              53
prevalentStroke      0
prevalentHyp         0
diabetes             0
totChol             50
sysBP                0
diaBP                0
BMI                 19
heartRate            1
glucose            388
TenYearCHD           0
dtype: int64

In [339]:
# Usar SKLearn para reemplazar los datos faltantes
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
imputer = imputer.fit(dataset.iloc[:, 0:15])
dataset.iloc[:, 0:15] = imputer.transform(dataset.iloc[:, 0:15])

dataset.isnull().sum()

male               0
age                0
education          0
currentSmoker      0
cigsPerDay         0
BPMeds             0
prevalentStroke    0
prevalentHyp       0
diabetes           0
totChol            0
sysBP              0
diaBP              0
BMI                0
heartRate          0
glucose            0
TenYearCHD         0
dtype: int64

##### Balanceo
Posteriormente, se determinó si los datos estaban balanceados. En este caso, como se puede observar, sí se cuenta con 644 paros cardiacos vs 3594 no paros cardiacos. Por tal razón, se prosiguió a balancear los mismos.

In [291]:
#Calcular cuantas columans son paro y cuales no
print('Número de paros cardiacos:',dataset[dataset['TenYearCHD'] == 1].shape[0])
print('Número de no paros cardiacos:',dataset[dataset['TenYearCHD'] == 0].shape[0])
data2 = dataset.dropna(subset=['TenYearCHD'])

Número de paros cardiacos: 644
Número de no paros cardiacos: 3594


In [292]:
# Separar las características y la variable objetivo
X_temp = data2.drop('TenYearCHD', axis=1)
y_temp = data2['TenYearCHD']

#Aplicar SMOTE
smote = SMOTE()
X_resampled, y_resampled = smote.fit_resample(X_temp, y_temp)

dataset2 = pd.concat([X_resampled, y_resampled], axis=1)
print('Número de paros cardiacos balanceados:',dataset2[dataset2['TenYearCHD'] == 1].shape[0])
print('Número de no paros cardiacos balanceados:',dataset2[dataset2['TenYearCHD'] == 0].shape[0])

Número de paros cardiacos balanceados: 3594
Número de no paros cardiacos balanceados: 3594


##### Escalar
En este caso, se utilizó la librería de SKLearn para poder escalar los datos. Esto con el propósito de tener datos en un rango de valores similares para que sea más fácil el análisis de los mismos.

In [293]:
scaler = MinMaxScaler()
scaled_features = scaler.fit_transform(dataset2)
df_feat = pd.DataFrame(scaled_features, columns=dataset2.columns)
df_feat.head()

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1.0,0.184211,1.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.149406,0.106383,0.232804,0.277024,0.363636,0.104520,0.0
1,0.0,0.368421,0.333333,0.0,0.000000,0.0,0.0,0.0,0.0,0.242784,0.177305,0.349206,0.319680,0.515152,0.101695,0.0
2,1.0,0.421053,0.000000,1.0,0.285714,0.0,0.0,0.0,0.0,0.234295,0.208038,0.338624,0.237518,0.313131,0.084746,0.0
3,0.0,0.763158,0.666667,1.0,0.428571,0.0,0.0,1.0,0.0,0.200340,0.314421,0.497354,0.316045,0.212121,0.177966,1.0
4,0.0,0.368421,0.666667,1.0,0.328571,0.0,0.0,0.0,0.0,0.302207,0.219858,0.380952,0.183228,0.414141,0.127119,0.0


##### Selección de Variables

Finalmente, se quiso determinar los features que se estarían utilizando para la realización del modelo. En este caso, se seleccionaron las variables con mayor correlación con TenYearCHD. 

In [294]:
# Correlación entre las columnas y el status, valores absolutos
corr = df_feat.corr()['TenYearCHD'].abs().sort_values(ascending=False)
print(corr)

# Elegir variables con correlación mayor a 0.1. 
corr = corr[corr > 0.1]

# Crear un nuevo dataframe con las variables seleccionadas
df_feat = df_feat[corr.index]

# Separar las características y la variable objetivo
X = df_feat.drop('TenYearCHD', axis=1)
y = df_feat['TenYearCHD']

TenYearCHD         1.000000
age                0.326011
sysBP              0.276992
prevalentHyp       0.243732
diaBP              0.201807
male               0.113976
glucose            0.113912
BPMeds             0.111842
totChol            0.108644
BMI                0.106629
diabetes           0.084914
prevalentStroke    0.075123
education          0.073690
cigsPerDay         0.053814
heartRate          0.028201
currentSmoker      0.009025
Name: TenYearCHD, dtype: float64


_______________

## Tarea 1.2
#### Ajustar un modelo logístico polinomial en base al juego de datos cargado de forma matricial que relaciona las variables independientes que usted considere apropiadas (puede no utilizar todas las componentes de X), con la variable dependiente de salida (sufre o no sufre un paro cardíaco).


En este caso, se creó primero (a través de PolynomialFeatures) la primera parte del modelo polinomial donde se indica el grado 2 (cuadrático). Posteriormente, se separaron los datos en entrenamiento (80%) y prueba (20%). En base a esto, se presenta la matriz de confusión y el accuracy. 

En este caso, se presenta un accuracy de 74%. Ya que no es perfecto, no se corre el riesgo de overfitting.

Cabe destacar que se creó el modelo sin realizar balanceo, ni scaling. Se obtuvo un peor accuracy. 

In [334]:
# Modelo logistico polinomial
poly = PolynomialFeatures(degree=2)
X_poly = poly.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_poly, y, test_size=0.2, random_state=42)

# Crear el modelo
logreg = LogisticRegression(max_iter=1000)
logreg.fit(X_train, y_train)

# Predecir con el modelo
y_pred = logreg.predict(X_test)

# Calcular la matriz de confusión
print('Matriz de confusión: ', confusion_matrix(y_test, y_pred))

# Calcular la accuracy del modelo
print('Accuracy del modelo:', accuracy_score(y_test, y_pred))

Matriz de confusión:  [[590 162]
 [203 483]]
Accuracy del modelo: 0.7461752433936022


_______________

## Tarea 1.3
#### Utilice la implementación vectorial del algoritmo de regresión logística (descenso del gradiente visto en clase).

En este caso, se hizo uso de SGDClassifier que permite realizar una regresión logística utilizando el descenso de gradiente. Cabe destacar que se obtuvo un accuracy bastante similar al modelo anterior, así confirmando el procedimiento actualmente realizado.

In [335]:
# Descenso de gradiente. 
# SGDClassifier tiene como parámetro loss='log_loss' para regresión logística. Implementa descenso de gradiente estocástico.
sgd = SGDClassifier(loss='log_loss', max_iter=1000, tol=1e-3)

sgd.fit(X_train, y_train)
y_pred = sgd.predict(X_test)
print('Matriz de confusión: ', confusion_matrix(y_test, y_pred))

# Calcular la accuracy del modelo
print('Accuracy del modelo:', accuracy_score(y_test, y_pred))

Matriz de confusión:  [[584 168]
 [198 488]]
Accuracy del modelo: 0.7454798331015299


_______________

## Tarea 1.4 
#### Usando cross-validation determine el grado del polinomio que mejor describe la nube de puntos (encuentre el mejor balance entre apego a los datos de entrenamiento y generalización para datos previamente no observados).

En este caso, a través de Cross Validation se determinó que el mejor grado del polinomio es 5. Al realizar el respectivo modelo posteriormente con Descenso de Gradiente, se puede observar un mejor rendimiento a comparación de cuando se estaba usando el polinomio de grado 2.

In [337]:
# cross validation

# Determinar mejor grado de polinomio del 1 al 5
polinomio_grado_accuracy = [[], [], [], [], []]

for i in range(1, 6):
    poly = PolynomialFeatures(degree=i)
    X_poly = poly.fit_transform(X)

    X_train, X_test, y_train, y_test = train_test_split(X_poly, y, test_size=0.2, random_state=23)

    sgd = SGDClassifier(loss='log_loss', max_iter=1000, tol=1e-3)
    sgd.fit(X_train, y_train)

    # Cross Validation de SKLearn. Se le manda modelo entrenado con datos de training y también se manda
    # datos de prueba para determinar rendimiento.
    cross_val = cross_val_score(sgd, X_test, y_test, cv=5)

    polinomio_grado_accuracy[i-1].append(i)
    polinomio_grado_accuracy[i-1].append(cross_val.mean())

# mostrar mejor grado de polinomio
print('\nEl mejor grado de polinomio es:', max(polinomio_grado_accuracy, key=lambda x: x[1])[0])


El mejor grado de polinomio es: 5


Como se puede observar, al utilizar el grado de polinomio recomendado por Cross Validation se obtuvo un accuracy mayo de 77%. 

In [338]:
# Modelo logistico polinomial
poly = PolynomialFeatures(degree=5)
X_poly = poly.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_poly, y, test_size=0.2, random_state=23)

# Crear el modelo
logreg = LogisticRegression(max_iter=1000)
logreg.fit(X_train, y_train)

# Predecir con el modelo
y_pred = logreg.predict(X_test)

# Calcular la matriz de confusión
print('Matriz de confusión: ', confusion_matrix(y_test, y_pred))

# Calcular la accuracy del modelo
print('Accuracy del modelo:', accuracy_score(y_test, y_pred))

Matriz de confusión:  [[585 109]
 [218 526]]
Accuracy del modelo: 0.7726008344923505


_______________

## Tarea 1.5 
#### Haga un análisis sobre sus hallazgos.